In [ ]:
pip install pandas numpy scikit-learn scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611316 sha256=1387dfa1b5f8a3cdca4ba1def0748501a5e08d6d13226cf97a2cafb2cdbd3842
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


In [ ]:
import pandas as pd
#                                                                                           1. Chargement du dataset MovieLens 100k
# Charger les données
ratings_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
movies_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.item"

# Chargement des notes
df_ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Chargement des films
movies_df = pd.read_csv(movies_url, sep='|', encoding='latin-1', names=[
    'movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
    'unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
    'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'
])

# Afficher les premières lignes
print(df_ratings.head())
print(movies_df[['movie_id', 'title', 'Action', 'Comedy', 'Drama']].head())


   user_id  movie_id  rating  timestamp
0      196       242       3  881250949
1      186       302       3  891717742
2       22       377       1  878887116
3      244        51       2  880606923
4      166       346       1  886397596
   movie_id              title  Action  Comedy  Drama
0         1   Toy Story (1995)       0       1      0
1         2   GoldenEye (1995)       1       0      0
2         3  Four Rooms (1995)       0       0      0
3         4  Get Shorty (1995)       1       1      1
4         5     Copycat (1995)       0       0      1


In [2]:
# ===============================================================
# 1️⃣ Installation propre et compatible
# ===============================================================
!pip install --quiet numpy==1.26.4 scikit-surprise==1.1.3

# ===============================================================
# 2️⃣ Import des librairies
# ===============================================================
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy

# ===============================================================
# 3️⃣ Création d’un petit jeu de données (ou remplace par ton fichier)
# ===============================================================
data = {
    'user_id': [1, 1, 1, 2, 2, 3, 3, 4, 4, 5],
    'movie_id': [10, 20, 30, 10, 30, 20, 30, 10, 20, 30],
    'rating': [5, 4, 3, 4, 5, 2, 4, 5, 3, 4]
}
df_ratings = pd.DataFrame(data)

# ===============================================================
# 4️⃣ Préparation des données pour Surprise
# ===============================================================
reader = Reader(rating_scale=(1, 5))
data_surprise = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)

# Split en train/test
trainset, testset = train_test_split(data_surprise, test_size=0.25, random_state=42)

# ===============================================================
# 5️⃣ Implémentation du filtrage collaboratif USER-BASED
# ===============================================================
sim_options = {
    'name': 'cosine',  # Similarité cosinus
    'user_based': True  # Mettre False pour item-based
}

algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)

# ===============================================================
# 6️⃣ Évaluation du modèle
# ===============================================================
rmse = accuracy.rmse(predictions)

# ===============================================================
# 7️⃣ Exemple de prédiction manuelle
# ===============================================================
uid = str(1)     # ID utilisateur
iid = str(30)    # ID film
pred = algo.predict(uid, iid)
print(f"Prédiction de la note de l’utilisateur {uid} pour le film {iid} → {pred.est:.2f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.9548
Prédiction de la note de l’utilisateur 1 pour le film 30 → 3.71


In [3]:
# ===============================================================
# 1️⃣ Charger les données MovieLens
# ===============================================================
import pandas as pd

ratings_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
movies_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.item"

# Notes
df_ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Films + genres
movies_df = pd.read_csv(movies_url, sep='|', encoding='latin-1', names=[
    'movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
    'unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
    'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'
])

# ===============================================================
# 2️⃣ Fusionner notes + films
# ===============================================================
merged_df = pd.merge(df_ratings, movies_df[['movie_id', 'title', 'Action','Adventure','Comedy','Drama','Romance','Sci-Fi','Thriller']], on='movie_id')

# ===============================================================
# 3️⃣ Choisir un utilisateur et trouver ses genres préférés
# ===============================================================
user_id = 5  # Tu peux changer l’utilisateur ici

# Notes de l'utilisateur
user_ratings = merged_df[merged_df['user_id'] == user_id]

# Genres pondérés par les notes
genre_columns = ['Action','Adventure','Comedy','Drama','Romance','Sci-Fi','Thriller']
genre_scores = user_ratings[genre_columns].multiply(user_ratings['rating'], axis=0).sum()

# Trier les genres préférés
favorite_genres = genre_scores.sort_values(ascending=False)
print("🎬 Genres préférés de l’utilisateur", user_id)
print(favorite_genres)

# ===============================================================
# 4️⃣ Recommandation : films non vus correspondant aux genres préférés
# ===============================================================
# Films déjà notés par l’utilisateur
watched = user_ratings['movie_id'].unique()

# Films non vus
unwatched = movies_df[~movies_df['movie_id'].isin(watched)]

# Calcul d’un score basé sur les genres préférés
unwatched['score_genre'] = unwatched[genre_columns].dot(favorite_genres)

# Trier les recommandations
recommendations = unwatched.sort_values('score_genre', ascending=False).head(5)

# ===============================================================
# 5️⃣ Afficher les 5 films recommandés
# ===============================================================
print("\n🎯 Top 5 films recommandés pour l’utilisateur", user_id)
print(recommendations[['title', 'score_genre']])


🎬 Genres préférés de l’utilisateur 5
Comedy       246
Action       176
Sci-Fi       116
Adventure    107
Drama         72
Thriller      56
Romance       44
dtype: int64

🎯 Top 5 films recommandés pour l’utilisateur 5
                                     title  score_genre
183                Army of Darkness (1993)          645
1109                      Tank Girl (1995)          538
200                    Evil Dead II (1987)          529
559   Kid in King Arthur's Court, A (1995)          513
3                        Get Shorty (1995)          494


/tmp/ipython-input-2976579102.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unwatched['score_genre'] = unwatched[genre_columns].dot(favorite_genres)


In [4]:
# ===============================================================
# 1️⃣ Imports
# ===============================================================
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

# ===============================================================
# 2️⃣ Charger MovieLens 100k
# ===============================================================
ratings_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
movies_url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.item"

df_ratings = pd.read_csv(ratings_url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies_df = pd.read_csv(movies_url, sep='|', encoding='latin-1', names=[
    'movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url',
    'unknown','Action','Adventure','Animation','Children','Comedy','Crime','Documentary',
    'Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'
])

# ===============================================================
# 3️⃣ Filtrage collaboratif avec Surprise (user-based)
# ===============================================================
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'movie_id', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

sim_options = {'name': 'cosine', 'user_based': True}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

# ===============================================================
# 4️⃣ Recommandation basée sur le contenu (genres)
# ===============================================================
merged_df = pd.merge(df_ratings, movies_df, on='movie_id')

user_id = 5  # Exemple : utilisateur 5

# Genres
genre_cols = ['Action','Adventure','Comedy','Drama','Romance','Sci-Fi','Thriller']
user_ratings = merged_df[merged_df['user_id'] == user_id]

genre_scores = user_ratings[genre_cols].multiply(user_ratings['rating'], axis=0).sum()
favorite_genres = genre_scores / genre_scores.sum()

# Films non vus
watched = user_ratings['movie_id'].unique()
unwatched = movies_df[~movies_df['movie_id'].isin(watched)].copy()

# Score de genre
unwatched['score_genre'] = unwatched[genre_cols].dot(favorite_genres)

# ===============================================================
# 5️⃣ Prédiction du modèle collaboratif pour les films non vus
# ===============================================================
pred_scores = []
for movie_id in unwatched['movie_id']:
    pred = model.predict(user_id, movie_id)
    pred_scores.append(pred.est)

unwatched['score_cf'] = pred_scores

# ===============================================================
# 6️⃣ Combinaison des deux scores
# ===============================================================
# Moyenne pondérée (tu peux changer les poids)
alpha = 0.6  # poids du filtrage collaboratif
beta = 0.4   # poids du contenu
unwatched['score_final'] = alpha * unwatched['score_cf'] + beta * unwatched['score_genre']

# Top 5
recommendations = unwatched.sort_values('score_final', ascending=False).head(5)

# ===============================================================
# 7️⃣ Résultat final
# ===============================================================
print("🎯 Top 5 films recommandés pour l’utilisateur", user_id)
print(recommendations[['title', 'score_cf', 'score_genre', 'score_final']])


Computing the cosine similarity matrix...
Done computing similarity matrix.
🎯 Top 5 films recommandés pour l’utilisateur 5
                          title  score_cf  score_genre  score_final
1655         Little City (1998)       5.0     0.354957     3.141983
1301       Late Bloomers (1996)       5.0     0.301102     3.120441
1499  Santa with Muscles (1996)       5.0     0.301102     3.120441
1462           Boys, Les (1997)       5.0     0.301102     3.120441
1292            Star Kid (1997)       5.0     0.272950     3.109180
